In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI()

a = chat.predict("얼마나 많은 행성이 있어?")

a


'현재까지 발견된 행성은 수백억 개 이상이 있습니다. 그러나 인류가 직접적으로 관측할 수 있는 행성은 태양계 내의 8개 행성과 몇몇 외계 행성들뿐입니다. 외계 행성은 계속 발견되고 있으며, 추정에 따르면 은하계 전체에는 수십억에서 수조에 이를 정도로 많은 행성들이 존재할 것으로 예상됩니다.'